# PEFT (Parameter-Efficient Fine-Tuning)
## 前言：為什麼需要 PEFT？

學會了 SFT、DPO、PPO 等對齊方法的「目標函數」和「訓練邏輯」。但當你真正要在自己的機器上跑這些訓練時，會遇到一個殘酷的現實：

### 全參數 Fine-tuning 的資源需求

**以 LLaMA-2 7B 為例：**

- 模型參數量：7B (70億)
- 每個參數：2 bytes (fp16) 或 4 bytes (fp32)

**只是載入模型：**

- 模型權重: 7B × 2 bytes = 14 GB

**訓練時需要的記憶體：**

- 梯度: 7B × 2 bytes = 14 GB
- 優化器狀態 (AdamW): 7B × 8 bytes = 56 GB (m, v 各 4 bytes)
- 激活值 (activations): ~20-40 GB (視 batch size)
- 總計: 14 + 14 + 56 + 30 = 114 GB

**一張 A100 (80GB) 都不夠用**

**更大的模型更誇張：**

- LLaMA-2 13B: ~200 GB
- LLaMA-2 70B: ~1 TB
- GPT-3 175B: ~2.5 TB

**全參數訓練 70B 模型需要：**

- 8-16 張 A100 (80GB)
- 分散式訓練框架
- 幾萬到幾十萬美金的算力成本

### 問題不只是記憶體

1. 訓練時間長
    - 更新所有參數 → 每步更慢
    - 70B 模型一步可能要幾秒
2. 過擬合風險高
    - 可調參數太多 → 容易記住訓練資料
    - 特別是資料量小的時候
3. 部署困難
    - 每個任務要存一份完整模型
    - 10 個任務 = 10 × 14GB = 140GB
4. 訓練不穩定
    - 參數空間巨大 → 容易發散
    - 需要極小的學習率和精細調參

## PEFT 的核心理念

**核心理念：** 凍結絕大部分參數，只訓練極少量的新增參數

- Full Fine-tuning：更新 7B 參數
- PEFT：更新~10M 參數 (0.14%)
- 記憶體需求：114 GB → 20 GB
- 訓練速度：1x → 3-5x
- 儲存成本：14 GB/任務 → 10 MB/任務
- 效果損失：通常 < 5%

**LLM 的 fine-tuning 發生在低秩（low-rank）子空間中**

### 基本想法

原始模型：W ∈ R^(d×k)  (例如 4096×4096)

**Full Fine-tuning**： W_new = W + ΔW

- 其中 ΔW 的每個元素都可能改變

**PEFT**：W_new = W + low_rank_adaptation

- 其中 adaptation 只有很少的自由度

### 為什麼low-rank假設有效？
假設我們有一個已經訓練好的大模型，它的權重叫做 W_original。現在我們拿這個模型去做全參數 fine-tune，得到一組新的權重 W_finetuned。如果我們把兩者相減，得到的差值：
$$ΔW = W_{finetuned} − W_{original}$$

過去的實驗發現，這個 ΔW 雖然看起來是一個超大的矩陣（例如 4096×4096），理論上rank可以高達 4096，但實際上並不是這樣。你如果對 ΔW 做 SVD，會發現前面只有非常少數幾個奇異值就已經解釋了幾乎全部的變化量。像是前 16 個奇異值，就能吃掉 95% 以上的能量。

用白話講就是：模型在 fine-tune 的時候，真的有在學的新東西，其實集中在一個很低維的方向空間裡。它並沒有用滿整個 4096 維的自由度，而是只在其中一小撮「重要方向」上做調整。

這個觀察就是 LoRA 的核心直覺。既然真正有用的更新只活在一個低維子空間，那我們幹嘛還要傻傻地更新整個 4096×4096 的權重矩陣？參數量爆炸、VRAM又貴，完全沒必要。

**LoRA 的做法就是**：原本的權重 W_original 我們凍結不動，然後額外加上一個「低秩更新項」。這個更新項通常被拆成兩個小矩陣相乘，一個是把高維空間壓到低維，另一個再把低維投回原本的維度。這樣一來，我們實際學的參數量就只跟那個低維的 rank 有關，比如 8、16、32，而不是 4096²。

### PEFT 的統一框架
**所有 PEFT 方法都遵循相同的模式：**

In [ ]:
class PEFTMethod:
    def __init__(self, pretrained_model):
        self.backbone = pretrained_model
        self.backbone.requires_grad_(False)  # 凍結
        
        self.trainable_params = self.create_efficient_params()
    
    def forward(self, x):
        # 原始模型的輸出
        base_output = self.backbone(x)
        
        # PEFT 的調整
        adaptation = self.apply_efficient_params(x)
        
        # 組合
        return base_output + adaptation
    
    def create_efficient_params(self):
        """各種 PEFT 方法的差異在這裡"""
        raise NotImplementedError

## 主流 PEFT 方法分類
### 方法概覽

**Additive Methods (加法式)**
- Adapter

    └─ 在層之間插入小型可訓練模組
- Prefix Tuning

    └─ 在輸入層添加可訓練的前綴向量
- Prompt Tuning

    └─ 只在最前面加可訓練的 soft prompt

**Selective Methods (選擇式)**
- BitFit

    └─ 只訓練 bias 參數

- Partial Fine-tuning

    └─ 只訓練特定層（如最後幾層）

**Reparameterization Methods (重參數化)**
- LoRA (Low-Rank Adaptation)

    └─ 用低秩矩陣分解來近似 ΔW

### 方法比較表

| 方法 | 可訓練參數量 | 推理延遲 | 記憶體效率 | 效果 | 實作複雜度 |
|------|------------|---------|-----------|------|-----------|
| **LoRA** | 0.1-1% | 無 | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | 低 |
| **Adapter** | 0.5-2% | 有 | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | 中 |
| **Prefix Tuning** | 0.01-0.1% | 有 | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | 中 |
| **Prompt Tuning** | <0.01% | 有 | ⭐⭐⭐⭐⭐ | ⭐⭐ | 低 |

## LoRA：為什麼成為主流？
### 核心想法

**LoRA (Low-Rank Adaptation) 的想法極其簡單：**

**原始權重矩陣**： W ∈ R^(d×k)

**Full Fine-tuning**：W' = W + ΔW  (ΔW 是 d×k 的滿秩矩陣)

**LoRA**：W' = W + BA

其中：
- B ∈ R^(d×r)
- A ∈ R^(r×k)
- r << min(d, k)  (rank，通常 r=8,16,32)

**參數量比較：**
d=4096, k=4096, r=16

**Full**: d × k = 16,777,216 參數

**LoRA**: d × r + r × k = r(d + k) = 131,072 參數 (0.78%)

### 為什麼 LoRA 這麼受歡迎？

**優勢 1：零推理延遲**

LoRA 在訓練時會多學一組小矩陣，但在部署前可以直接把這些權重合併回原本的模型，只需要做一次合併就好。合併之後，推理時的計算流程和原本模型完全一樣，不會多出任何額外運算，因此幾乎沒有推理延遲。這一點跟 Adapter 很不一樣，Adapter 在推理時還得多走一次前向傳播，自然就會拖慢速度。

**優勢 2：模組化切換**

In [ ]:
# 多任務部署
base_model = load_pretrained()

# 載入不同任務的 LoRA 權重
lora_A_task1, lora_B_task1 = load_lora("task1")
lora_A_task2, lora_B_task2 = load_lora("task2")

# 快速切換
def switch_task(task_id):
    if task_id == 1:
        apply_lora(lora_A_task1, lora_B_task1)
    elif task_id == 2:
        apply_lora(lora_A_task2, lora_B_task2)


# 儲存空間
"""
Base model: 14 GB
LoRA per task: 10-50 MB
10 個任務: 14 GB + 10 × 50 MB = 14.5 GB
vs Full FT: 10 × 14 GB = 140 GB
"""

**優勢 3：訓練穩定**

LoRA 的初始化設計讓訓練一開始不會影響原本的預訓練權重，模型等於是從原狀態起跑，再慢慢學會怎麼調整。這樣可以避免訓練初期的不穩定，讓模型用更平滑、漸進的方式進行微調。

## LoRA 的超參數

### r (Rank)：核心超參數

**經驗法則：**
- 任務複雜度低（如分類）：r = 4, 8

- 一般對齊任務（SFT / DPO）：r = 16, 32

- 複雜領域適應：r = 64, 128

r 的選擇本質上是在模型表達能力與資源成本之間取捨。較大的 r 能提供更強的參數自由度，通常有助於提升任務效果，但同時會增加可訓練參數數量、記憶體佔用與訓練成本，並提高過擬合的風險；相反地，較小的 r 雖然效率高、穩定性好，但可能限制模型的學習能力。

實務上可先以 r=16 作為基準設定，快速建立 baseline。若效果不足，再逐步提升至 r=32 或 64 觀察增益；若發現驗證集表現下降或出現過擬合跡象，則可反向嘗試較小的 r（如 8 或 4），以提高泛化能力並降低資源消耗。

### α (LoRA Alpha)：縮放係數

LoRA 在前向傳播時，會在原始權重輸出之外加入一個低秩更新項，其形式為 output = W @ x + (α / r) × B @ A @ x。

其中 (α / r) 是用來控制 LoRA 更新對整體輸出的影響比例。

**α 的作用**：α（alpha）負責調整 LoRA 更新的整體強度，可視為低秩權重的全域縮放係數。即使 r 固定，透過改變 α，也能讓模型對新任務的適應程度變得更激進或更保守。

**常見設定：**
- α = r  → 縮放因子 = 1（標準設定）

- α = 2r  → 縮放因子 = 2（較激進）

- α = r/2 → 縮放因子 = 0.5（較保守）

在大多數任務中，α=r 是穩定且效果可靠的預設選擇。若微調的是已對齊良好的模型、且希望模型更快速偏向新任務，可考慮提高 α；反之，若希望盡量保留原模型行為、僅做小幅調整，則降低 α 有助於控制變化幅度。


### Target Modules：應用 LoRA 的位置

Target Modules 指的是在模型中實際套用 LoRA 的權重矩陣位置。由於 LoRA 並不需要對所有參數進行微調，因此透過選擇性地指定 target modules，可以在「效能、參數量與訓練成本」之間取得平衡。不同模組負責的功能不同，對最終行為的影響程度也有所差異，這使得 target modules 的選擇成為實務中非常重要的設計決策。



In [ ]:
# Transformer 的權重矩陣
attention_weights = {
    'q_proj': Query 投影,
    'k_proj': Key 投影,
    'v_proj': Value 投影,
    'o_proj': Output 投影,
}

ffn_weights = {
    'gate_proj': FFN 第一層,
    'up_proj': FFN 第二層,
    'down_proj': FFN 輸出層,
}

# 常見配置

# 配置 1: 只訓練 Q, V (最省資源)
target_modules = ['q_proj', 'v_proj']
# 參數量: 最少
# 效果: 中等
# 適用: 資源極度受限

# 配置 2: 訓練所有 attention (標準)
target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj']
# 參數量: 中
# 效果: 好
# 適用: 大多數場景

# 配置 3: Attention + FFN (最全面)
target_modules = [
    'q_proj', 'k_proj', 'v_proj', 'o_proj',
    'gate_proj', 'up_proj', 'down_proj'
]
# 參數量: 最多
# 效果: 最好
# 適用: 需要強領域適應

## LoRA 實作範例

### 載入基礎模型

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto"
)

### 配置 LoRA

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    
    # LLaMA 常見設定
    target_modules=["q_proj", "v_proj"],
    
    lora_dropout=0.05,
    bias="none",                        # bias 訓練策略
    task_type=TaskType.CAUSAL_LM,       # 任務類型
    inference_mode=False,
)

### 訓練 LoRA


In [ ]:
model = get_peft_model(base_model, lora_config)
model.train()

model.print_trainable_parameters()

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora_output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,                 # LoRA 可用較大學習率
    logging_steps=10,
    save_steps=500,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,  # Causal LM 常見
)

trainer.train()
model.save_pretrained("./lora_weights")

### 載入 LoRA 進行推理

In [ ]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "./lora_weights")
model.eval()

# 推理
output = model.generate(...)


# (可選) 合併權重以加速推理
#model = model.merge_and_unload()

## 其他 PEFT 方法簡介
### Adapter

**核心想法：** 在 Transformer 的每一層插入小型可訓練的瓶頸模組

![IMG](https://truth.bahamut.com.tw/s01/202602/47de79fd93b79f888bb304db164dfee7.PNG)

**Adapter 的結構：**


In [ ]:
class Adapter(nn.Module):
    def __init__(self, d_model, bottleneck_dim):
        super().__init__()
        # 降維
        self.down_proj = nn.Linear(d_model, bottleneck_dim)
        self.activation = nn.ReLU()
        # 升維
        self.up_proj = nn.Linear(bottleneck_dim, d_model)
    
    def forward(self, x):
        # 瓶頸結構
        h = self.down_proj(x)       # d_model → bottleneck
        h = self.activation(h)
        h = self.up_proj(h)         # bottleneck → d_model
        return x + h                # 殘差連接

**參數量**
- d_model = 4096

- bottleneck = 256

- 參數量 = 4096×256 + 256×4096 = 2,097,152  (每個 adapter)

**優缺點：**

- 簡單直觀
- 靈活（可調整瓶頸大小）
- 推理時有額外計算（不像 LoRA 可合併）
- 參數量比 LoRA 略多

### Prefix Tuning

**核心想法：** 在每層的 Key 和 Value 前面添加可訓練的前綴向量


**原始 Attention：**
- Q = Input × W_q

- K = Input × W_k

- V = Input × W_v

- Attention(Q, K, V)

**Prefix Tuning：**

- Q = Input × W_q

- K = [Prefix_K; Input × W_k]  ← 拼接可訓練前綴

- V = [Prefix_V; Input × W_v]  ← 拼接可訓練前綴

- Attention(Q, K, V)

**實作：**

In [ ]:
class PrefixTuning(nn.Module):
    def __init__(self, num_layers, num_heads, d_head, prefix_len):
        super().__init__()
        # 每層都有自己的 prefix
        self.prefix_k = nn.Parameter(
            torch.randn(num_layers, prefix_len, num_heads, d_head)
        )
        self.prefix_v = nn.Parameter(
            torch.randn(num_layers, prefix_len, num_heads, d_head)
        )
    
    def forward(self, layer_idx, k, v):
        # k, v: [batch, seq_len, num_heads, d_head]
        prefix_k = self.prefix_k[layer_idx].unsqueeze(0)  # [1, prefix_len, ...]
        prefix_v = self.prefix_v[layer_idx].unsqueeze(0)
        
        # 拼接
        k = torch.cat([prefix_k.expand(k.size(0), -1, -1, -1), k], dim=1)
        v = torch.cat([prefix_v.expand(v.size(0), -1, -1, -1), v], dim=1)
        
        return k, v

**優缺點：**
- 參數量極少（< 0.1%）
- 不改變模型結構
- 推理時佔用序列長度
- 效果通常不如 LoRA
- 訓練初期不穩定

### Prompt Tuning

**核心想法：** 只在輸入層添加可訓練的 soft prompts

**原始輸入：**[User instruction tokens]

**Prompt Tuning：**[Learnable soft prompt tokens] + [User instruction tokens]

**例子：**
- 原始: "Translate to French: Hello"
- PT:   [v1][v2][v3]...[v10] + "Translate to French: Hello"

其中 v1...v10 是可學習的向量，不對應任何真實 token

**實作：**

In [ ]:
class PromptTuning(nn.Module):
    def __init__(self, num_prompts, embedding_dim):
        super().__init__()
        # 可訓練的 soft prompt embeddings
        self.soft_prompt = nn.Parameter(
            torch.randn(num_prompts, embedding_dim)
        )
    
    def forward(self, input_embeds):
        # input_embeds: [batch, seq_len, embedding_dim]
        batch_size = input_embeds.size(0)
        
        # 擴展 soft prompt 到 batch
        prompts = self.soft_prompt.unsqueeze(0).expand(batch_size, -1, -1)
        
        # 拼接到輸入前面
        return torch.cat([prompts, input_embeds], dim=1)

**優缺點：**

- 參數量最少（幾 KB）

- 實作最簡單

- 效果最差（尤其是小模型）

- 需要大模型（>10B）才有效

- 只適合特定類型任務

## LoRA 超參數推薦表

### 根據任務類型

任務類型          | r    | α    | target_modules           | dropout
----------------|------|------|-------------------------|--------
分類/標註         | 8    | 16   | q_proj, v_proj          | 0.05
QA/抽取式         | 16   | 32   | q,k,v,o_proj            | 0.05
對話/指令遵循     | 16   | 32   | q,k,v,o_proj            | 0.05
摘要              | 32   | 64   | q,k,v,o + FFN           | 0.1
程式碼生成        | 64   | 128  | 全部 attention + FFN     | 0.1
數學推理          | 64   | 128  | 全部 attention + FFN     | 0.05
多語言            | 128  | 256  | 全部 attention + FFN     | 0.1
領域適應 (醫學等) | 64   | 128  | 全部 attention + FFN     | 0.1


### 根據模型大小

模型大小      | r (起始) | 可嘗試範圍
-----------|---------|-------------
< 1B       | 4-8     | 4-16
1B-7B      | 16      | 8-64
7B-13B     | 16-32   | 16-128
13B-70B    | 32      | 16-128
&gt; 70B      | 32-64   | 32-256


### 根據資料量


訓練資料量   | r    | 建議
----------|------|----------------------------------
< 1K      | 4-8  | 極小 r，避免過擬合
1K-10K    | 8-16 | 標準設定
10K-100K  | 16-32| 可適度增大
&gt; 100K    | 32-64| 可用較大 r，類似 Full FT


### 學習率設定

LoRA 可以用比 Full FT 更大的學習率

訓練方法        | 學習率範圍        | 推薦起始值
-------------|-----------------|------------
Full FT      | 5e-6 ~ 5e-5     | 2e-5
LoRA (r≤16)  | 1e-4 ~ 5e-4     | 2e-4
LoRA (r>32)  | 5e-5 ~ 2e-4     | 1e-4
Adapter      | 1e-4 ~ 1e-3     | 5e-4
Prefix Tuning| 5e-5 ~ 5e-4     | 1e-4
